In [10]:
import pandas as pd
tensorflow_commits = pd.read_csv('C:/Users/aveli/Downloads/tensorflow.csv')
vscode_commits=pd.read_csv('C:/Users/aveli/Downloads/vscode.csv')
react_commits=pd.read_csv('C:/Users/aveli/Downloads/react-native.csv')

total_commits=tensorflow_commits.append(vscode_commits, ignore_index=True)
total_commits=total_commits.append(react_commits, ignore_index=True)
                             
total_commits.shape
                             

#total_commits.shape

(709762, 21)

In [11]:
# Use h2o4gpu if you have it installed

# Add a flag with Default as False. Don't change this unless your kernel uses h2o4gpu.
h2o4gpu_enabled = False

In [12]:
#Creating various features for each modification
# Total number of lines changed
total_commits['total_changed'] = total_commits['lines_added'] + total_commits['lines_removed']

# Fraction of lines changed per total numbe of lines in file
# We need to account for the fact that new files added with have existing size as '0' and divide by '0' is indeterminate
total_commits['size'].loc[total_commits['size'] == 0] = total_commits['total_changed']
total_commits['ratio_changed'] = total_commits['total_changed'] / total_commits['size']

# Need to weigh the complexity by quantum of change. 
total_commits['rated_complexity'] = total_commits['ratio_changed'] * total_commits['complexity'] * total_commits['total_changed']

# weighing the dmm params by the total changed lines
total_commits['total_dmm_size'] = total_commits['total_changed'] * total_commits['dmm_unit_size']
total_commits['total_dmm_unit_complexity'] = total_commits['total_changed'] * total_commits['dmm_unit_complexity']
total_commits['total_dmm_unit_interfacing'] = total_commits['total_changed'] * total_commits['dmm_unit_interfacing']

# We picked the sqrt of no_of_mod_files to reduce weightage of this feature
total_commits['scaled_rated_complexity']=total_commits['rated_complexity'] * (total_commits['no._of_mod_files'] ** 0.5)

total_commits.shape

c:\users\aveli\python37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


(709762, 28)

In [13]:
# Preprocessing the data. ML requires the data to be converted to numericals
#ml_commits = total_commits[['hash','Author', 'no._of_mod_files', 'dmm_unit_size',
#       'dmm_unit_complexity', 'dmm_unit_interfacing', 'complexity', 'functions', 'lines_added', 'lines_removed', 
#       'tokens', 'type']]

ml_commits = total_commits[['hash','Author','total_changed','rated_complexity',
                            'total_dmm_unit_complexity','total_dmm_size','total_dmm_unit_interfacing', 'scaled_rated_complexity']]

# Resetting the frame's index. It is required to retain the integrity of the frame
ml_commits = ml_commits.reset_index().drop(columns = 'index')

# Temporarily dropping text columns for numeric processing
ml_commits_noText = ml_commits.drop(columns = ['Author','hash'])

# Explicitely converting fields to numeric types and filling the NaNs with zeros
ml_commits_numeric = ml_commits_noText.apply(pd.to_numeric,errors ='coerce').fillna(0)

# Adding the Author column back to create a 'total' data frame
ml_commits_total = ml_commits_numeric.copy()
ml_commits_total['Author'] = ml_commits['Author']
ml_commits_total['hash'] = ml_commits['hash']

print(ml_commits_total.shape)

(709762, 8)


In [14]:
# Attempt to remove outliers. May not be required
from scipy import stats
import numpy as np
# Calculate z_scores (and if zscore is greater than '3', then its an outlier) and collect normal subset.
ml_commits_nout = ml_commits_total[(np.abs(stats.zscore(ml_commits_total.select_dtypes(exclude='object'))) < 3).all(axis=1)]
ml_commits_nout.to_csv('C:/Users/aveli/Downloads/totalCommits_nout.csv')

# Collect outliers
ml_commits_out = ml_commits_total[~(np.abs(stats.zscore(ml_commits_total.select_dtypes(exclude='object'))) < 3).all(axis=1)]
ml_commits_out.to_csv('C:/Users/aveli/Downloads/totalCommits_out.csv')


In [15]:
# Applying scaler to regular data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
if (h2o4gpu_enabled == True):
    from h2o4gpu.preprocessing import MinMaxScaler
else:
    from sklearn.preprocessing import MinMaxScaler
    
# Use minMax scaler since this does not distort
# https://towardsdatascience.com/scale-standardize-or-normalize-with-scikit-learn-6ccc7d176a02
scaler = MinMaxScaler()
ml_commits_nout_numeric = ml_commits_nout.drop(columns = ['Author','hash'])
data_scaled = scaler.fit_transform(ml_commits_nout_numeric)

In [16]:
# Try and figure out how many clusters are optimum
# Plot the inertia curve to find the 'elbow'

''' Realised that h2o4GPU has not implemented "Intertia". Hence we will have to use regular Kmeans library for intertia. 
Please ignore this cell entirely.


import h2o4gpu

SSE = []

for cluster in range(1,20):
    kmeans_ss = h2o4gpu.KMeans(n_gpus=1, n_clusters = cluster, init='k-means++', random_state = 42, backend=h2o4gpu)
    %time kmeans_ss.fit(data_scaled)
    SSE.append(kmeans_ss.inertia_)
    print(kmeans_ss.cluster_centers_)
    print(len(SSE))
    print(str(SSE))
    
    # converting the results into a dataframe and plotting them
frame = pd.DataFrame({'Cluster':range(1,20), 'SSE':SSE})
plt.figure(figsize=(12,6))
plt.plot(frame['Cluster'], frame['SSE'], marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
'''

' Realised that h2o4GPU has not implemented "Intertia". Hence we will have to use regular Kmeans library for intertia. \nPlease ignore this cell entirely.\n\n\nimport h2o4gpu\n\nSSE = []\n\nfor cluster in range(1,20):\n    kmeans_ss = h2o4gpu.KMeans(n_gpus=1, n_clusters = cluster, init=\'k-means++\', random_state = 42, backend=h2o4gpu)\n    %time kmeans_ss.fit(data_scaled)\n    SSE.append(kmeans_ss.inertia_)\n    print(kmeans_ss.cluster_centers_)\n    print(len(SSE))\n    print(str(SSE))\n    \n    # converting the results into a dataframe and plotting them\nframe = pd.DataFrame({\'Cluster\':range(1,20), \'SSE\':SSE})\nplt.figure(figsize=(12,6))\nplt.plot(frame[\'Cluster\'], frame[\'SSE\'], marker=\'o\')\nplt.xlabel(\'Number of clusters\')\nplt.ylabel(\'Inertia\')\n'

In [17]:
# Try and figure out how many clusters are optimum
# Plot the inertia curve to find the 'elbow'
# You don't need to run this for every iteration. Just uncomment and run whenever you need to

# Realised that h2o4GPU has not implemented "Intertia". Hence we will have to use regular Kmeans library for intertia.
# http://docs.h2o.ai/h2o4gpu/latest-stable/h2o4gpu-py-docs/html/_modules/h2o4gpu/solvers/kmeans.html

# Set this to 'True' if you want to plot graph to find elbow
find_elbow = False

if (find_elbow == True): 
    from sklearn.cluster import KMeans

    SSE = []

    for cluster in range(1,20):
        %time kmeans_ss = KMeans(n_clusters = cluster, init='k-means++', random_state = 42)
        kmeans_ss.fit(data_scaled)
        SSE.append(kmeans_ss.inertia_)

    # converting the results into a dataframe and plotting them
    frame = pd.DataFrame({'Cluster':range(1,20), 'SSE':SSE})
    plt.figure(figsize=(12,6))
    plt.plot(frame['Cluster'], frame['SSE'], marker='o')
    plt.xlabel('Number of clusters')
    plt.ylabel('Inertia')

In [18]:
# Instantiating the KMeans object based on number discovered above
k = 5

if (h2o4gpu_enabled == True):
    import h2o4gpu
    kmeans = h2o4gpu.KMeans(n_gpus=1, n_clusters = k, init='k-means++', max_iter = 20, random_state = 42, backend=h2o4gpu)
else:
    from sklearn.mixture import GaussianMixture
    mix = GaussianMixture(n_components=k,random_state=42)
# Learning the Gaussian mixture model from data   
mix.fit(data_scaled)
# Saving the parameters of Gaussian mixture model in a file
import pickle
vfilename = 'C:/Users/aveli/Downloads/gmmclust_model.sav'
pickle.dump(mix, open(vfilename, 'wb'))
# Predicting the cluster labels of the data for the Gaussian mixture model
cluster_frame = pd.DataFrame(data_scaled)
gmmhash_clusters = mix.predict(cluster_frame)
# Collecting the mean of the Gaussian mixture model in 'gmmcentroids'
gmmcentroids = mix.means_

combinedCentroids = gmmcentroids[gmmhash_clusters].sum(axis=1)
# adding column with combined centroid values to the original dataframe 
ml_commits_nout['center'] = combinedCentroids
#print(combinedCentroids)
# Creating a dictionary with combined centroid values and target cluster labels
unique_centroids = np.unique(combinedCentroids).tolist()
cluster_labels = np.arange(5).tolist()
cluster_dict = dict(zip(unique_centroids,cluster_labels))
#print(g)
ml_commits_nout['fixed_cluster'] = ml_commits_nout['center'].map(cluster_dict)
# Converting the input data series into pan
ml_commits_nout['Cluster'] = gmmhash_clusters
#ml_commits_nout[ml_commits_nout['Cluster']==0]
#ml_commits_nout['sum_value'] = ml_commits_nout['scaled_rated_complexity']+ml_commits_nout['total_dmm_unit_interfacing']+ ml_commits_nout['total_dmm_size']+ml_commits_nout['total_dmm_unit_complexity']+ml_commits_nout['rated_complexity']+ml_commits_nout['ratio_changed']+ml_commits_nout['total_changed']
#ml_commits_nout[ml_commits_nout['Cluster']==3]  
ml_commits_nout

c:\users\aveli\python37\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\aveli\python37\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\aveli\python37\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

,total_changed,rated_complexity,total_dmm_unit_complexity,total_dmm_size,total_dmm_unit_interfacing,scaled_rated_complexity,Author,hash,center,fixed_cluster,Cluster
0,14,13.177872,14.000000,7.175000,7.525000,26.355745,A. Unique TensorFlower,0ef962b1a5b4e80a7029b0b159af6817b12a04df,0.004192,0,0
1,75,573.979592,75.000000,38.437500,40.312500,1147.959184,A. Unique TensorFlower,0ef962b1a5b4e80a7029b0b159af6817b12a04df,0.044030,2,3
2,4,0.301887,4.000000,2.050000,2.150000,0.603774,A. Unique TensorFlower,0ef962b1a5b4e80a7029b0b159af6817b12a04df,0.004192,0,0
3,47,266.518875,47.000000,24.087500,25.262500,533.037750,A. Unique TensorFlower,0ef962b1a5b4e80a7029b0b159af6817b12a04df,0.044030,2,3
4,19,58.992683,18.050000,18.050000,18.050000,117.985366,George Karpenkov,ef47bbbd57cba8fcc7ae11df8c7141d6c68ba0d0,0.044030,2,3
...,...,...,...,...,...,...,...,...,...,...,...
709757,175,0.000000,133.586576,99.679797,156.212240,0.000000,Ben Alpert,a15603d8f1ecdd673d80be318293cee53eb4475d,0.172553,3,1
709758,41,246.000000,31.297426,23.353552,36.598296,4808.025790,Ben Alpert,a15603d8f1ecdd673d80be318293cee53eb4475d,0.044030,2,3
709759,157,157.000000,119.846242,89.427018,140.144696,3068.536785,Ben Alpert,a15603d8f1ecdd673d80be318293cee53eb4475d,0.172553,3,1
709760,574,33866.000000,438.163968,326.949734,512.376148,661904.883795,Ben Alpert,a15603d8f1ecdd673d80be318293cee53eb4475d,0.764767,4,2


In [19]:
# Look at the values of inverted scaling of centroids for sanity
real_centroids = scaler.inverse_transform(gmmcentroids)

# Write these to dataframe
real_centroids_dataFrame = pd.DataFrame(real_centroids, columns=['total_changed','rated_complexity',
                            'total_dmm_unit_complexity','total_dmm_size','total_dmm_unit_interfacing', 'scaled_rated_complexity'])

# Add a cloumn for summing all coloumns
real_centroids_dataFrame['Sum_centroids'] = real_centroids_dataFrame.sum(axis = 1)

#You can write it out as csv if required.
real_centroids_dataFrame.to_csv('C:/Users/aveli/Downloads/totalCommits_centroids.csv')

In [20]:
# ml_commits_nout.columns

In [21]:
# Train the classifier
from xgboost import XGBClassifier
if (h2o4gpu_enabled == True):
    import h2o4gpu as sklearn
else:
    import sklearn

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

ml_commits_nout_numeric_xg = ml_commits_nout.drop(columns=['Author','hash','center','fixed_cluster'])


# Prepare the 'X' and 'Y' for the model
X_ml_commits_nout_numeric_xg = ml_commits_nout_numeric_xg.drop(columns = ['Cluster'])
Y_ml_commits_nout_numeric_xg = ml_commits_nout_numeric_xg['Cluster']

# Split the data for 'Training' and 'Testing' datasets
X_train, X_test, y_train, y_test = train_test_split(X_ml_commits_nout_numeric_xg, Y_ml_commits_nout_numeric_xg, random_state=7)

xgboostmodel = XGBClassifier()
# Training the xgboost classifier
xgboostmodel.fit(X_train, y_train)
# Predicting the class labels of test data for xgboost classifier
y_pred = xgboostmodel.predict(X_test)
 
# accuracy on X_test 
accuracy = accuracy_score(y_test, y_pred)
print('accuracy', accuracy)

accuracy 0.998419971559488


In [22]:
if (h2o4gpu_enabled == True):
    from h2o4gpu import metrics
else:
    from sklearn import metrics

metrics.silhouette_score(data_scaled,gmmhash_clusters)


0.554363566824674

In [23]:
import pickle
# Saving the parameters of the xgboost classifier in a file
filename = 'C:/Users/aveli/Downloads/finalized_model.sav'
pickle.dump(xgboostmodel, open(filename, 'wb'))